# Export RSCD Model for Mac Web Demo

This notebook loads the trained `rscd_resnet18.pt` model,
builds grouped predictions (Option B), and exports ONNX for a local web demo.

It does **not** modify `exploring.ipynb`.


In [8]:
from pathlib import Path
import numpy as np
import torch
from torch import nn
from torchvision import models, transforms, datasets

CHECKPOINT_PATH = Path('rscd_resnet18.pt')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMAGE_SIZE = 224

CHECKPOINT_PATH.exists()


True

In [10]:
# Build the same model architecture
def build_model(num_classes):
    model = models.resnet18(weights=None)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model

# Load checkpoint explicitly (avoids FutureWarning about default weights_only)
ckpt = torch.load(CHECKPOINT_PATH, map_location='cpu', weights_only=False)

# Checkpoint may be a dict with model_state + class_to_idx
if isinstance(ckpt, dict) and 'model_state' in ckpt:
    state_dict = ckpt['model_state']
    class_to_idx = ckpt.get('class_to_idx')
else:
    state_dict = ckpt
    class_to_idx = None

# Reconstruct class mapping if not in checkpoint (no image loading)
if class_to_idx is None:
    DATA_ROOT = Path('data') / 'RSCD dataset-1million'
    TRAIN_DIR = DATA_ROOT / 'train'
    train_ds = datasets.ImageFolder(TRAIN_DIR)
    class_to_idx = train_ds.class_to_idx

idx_to_class = {v: k for k, v in class_to_idx.items()}
num_classes = len(class_to_idx)

model = build_model(num_classes)
model.load_state_dict(state_dict)
model.to(DEVICE).eval()

num_classes


27

In [11]:
# Match eval transforms from training
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
eval_tfms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])


In [12]:
# Option B grouping: friction (dry/wet/water), surface, and winter types
FRICTION_CLASSES = ['dry', 'wet', 'water']
SURFACE_CLASSES = ['asphalt', 'concrete', 'gravel', 'mud']
WINTER_CLASSES = ['fresh_snow', 'melted_snow', 'ice']
UNEVEN_CLASSES = ['smooth', 'slight', 'severe']

def parse_groups(label_name):
    # label_name examples: dry_asphalt_smooth, wet_concrete_severe, melted_snow, water_mud
    parts = label_name.split('_')
    friction = None
    surface = None
    uneven = None
    winter = None

    # winter-only classes
    if label_name in WINTER_CLASSES:
        winter = label_name
        return friction, surface, uneven, winter

    # friction is always first for non-winter combined labels
    if parts and parts[0] in FRICTION_CLASSES:
        friction = parts[0]

    # detect surface and unevenness if present
    for p in parts[1:]:
        if p in SURFACE_CLASSES:
            surface = p
        elif p in UNEVEN_CLASSES:
            uneven = p

    return friction, surface, uneven, winter

# Build mapping from class index to groups
index_groups = {}
for idx, name in idx_to_class.items():
    index_groups[idx] = parse_groups(name)

index_groups[list(index_groups.keys())[0]]


('dry', 'asphalt', 'severe', None)

In [13]:
@torch.no_grad()
def predict_grouped(image_path: Path):
    img = datasets.folder.default_loader(image_path)
    x = eval_tfms(img).unsqueeze(0).to(DEVICE)
    logits = model(x)
    probs = torch.softmax(logits, dim=1).squeeze(0)

    # Aggregate probabilities by group
    friction_scores = {k: 0.0 for k in FRICTION_CLASSES}
    surface_scores = {k: 0.0 for k in SURFACE_CLASSES}
    winter_scores = {k: 0.0 for k in WINTER_CLASSES}
    uneven_scores = {k: 0.0 for k in UNEVEN_CLASSES}

    for idx, p in enumerate(probs):
        friction, surface, uneven, winter = index_groups[idx]
        if friction is not None:
            friction_scores[friction] += p.item()
        if surface is not None:
            surface_scores[surface] += p.item()
        if uneven is not None:
            uneven_scores[uneven] += p.item()
        if winter is not None:
            winter_scores[winter] += p.item()

    def topk(scores, k=3):
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]

    return {
        'friction': topk(friction_scores),
        'surface': topk(surface_scores),
        'uneven': topk(uneven_scores),
        'winter': topk(winter_scores),
    }

# Example (update to a real file path)
# sample = Path('data/RSCD dataset-1million/vali_20k')
# print(predict_grouped(next(sample.iterdir())))


In [14]:
# Export to ONNX for use in a Mac local web app
dummy = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE, device=DEVICE)
onnx_path = Path('rscd_resnet18.onnx')
torch.onnx.export(
    model,
    dummy,
    onnx_path.as_posix(),
    input_names=['input'],
    output_names=['logits'],
    dynamic_axes={'input': {0: 'batch'}, 'logits': {0: 'batch'}},
    opset_version=17,
)
onnx_path


WindowsPath('rscd_resnet18.onnx')